In [1]:
import json
import paramak
import neutronics_material_maker as nmm
from cad_to_h5m import cad_to_h5m
from copy import deepcopy
import random
import numpy as np
import openmc
import openmc_plasma_source
import math
import dagmc_h5m_file_inspector as di
import openmc_dagmc_wrapper as odw
import openmc_tally_unit_converter as otuc
import pandas
from spectrum_plotter import plot_spectrum_from_tally
from IPython.display import display

In [ ]:
rotation_angle=360

print('Enter filepath for JSON: ')
json_filename = input() #open json file
json_file = open(json_filename,)
data=json.load(json_file)
paramak_surfaces = []
max_r_coordinate_list = []
min_r_coordinate_list = []
max_r_coordinate_dict = {}
min_r_coordinate_dict = {}
material_tag_dict = {}
tf_coil_spacing = 360/data['tf']['coils_n']
tf_coil_locations = []
x=0
while x<data['tf']['coils_n']:  #extract number of tf coils
    tf_coil_locations.append(x*tf_coil_spacing) #calculate TF coil locations
    x+=1
for layer_dict in data['layer']:
    if 'outline' not in layer_dict:
        continue
    else:
        for key, value in layer_dict.items():
            if key == 'outline':
                value['r']=[in_meters*100 for in_meters in value['r']]  # convert to cm (necessary for neutronics)
                value['z']=[in_meters*100 for in_meters in value['z']]
                max_r_coordinate = max(value['r'])
                max_r_coordinate_list.append(max_r_coordinate) # finds max r coordinate for sorting layers
                max_r_coordinate_dict[layer_dict['name'].lower()] = max_r_coordinate
                min_r_coordinate = min(value['r'])
                min_r_coordinate_list.append(min_r_coordinate) #finds max z coordinate for sorting layers
                min_r_coordinate_dict[layer_dict['name'].lower()] = min_r_coordinate
                zip_coorinates = zip(value['r'],value['z'])
                coordinates_list = list(zip_coorinates) # makes list of r,z coordinates for defining geometry
                coordinates_list.pop(-1) # remove last point because first and last points are the same
                print(layer_dict['name'].lower() + ' max r: ' + str(max_r_coordinate))
                print(layer_dict['name'].lower() + ' min r: ' + str(min_r_coordinate))
                if 'hfs tf' in layer_dict['name'].lower():
                    for count, points in enumerate(coordinates_list):
                        temp_list = list(points)
                        if count == len(coordinates_list)-1: #spline shapes for everything except last and first points
                            temp_list.append('straight')
                        else:
                            temp_list.append('spline')
                        new_tuple = tuple(temp_list)
                        coordinates_list[count]=new_tuple
                    shape = paramak.ExtrudeMixedShape(
                        points = coordinates_list,
                        distance = data['tf']['thickness'] *100, # make into centimeters
                        azimuth_placement_angle= 
                            tf_coil_locations
                    )
                # elif len(coordinates_list) < 10: # # of points to distinguish simple vs spline shape, consider changing
                #     shape = paramak.RotateStraightShape(
                #     rotation_angle=rotation_angle,
                #     points = coordinates_list,
                #     )
                else:
                    shape = paramak.RotateStraightShape(
                        rotation_angle=rotation_angle,
                        points=coordinates_list,
                    )
        shape.name = layer_dict['name']
        shape.rotation_angle = rotation_angle
        paramak_surfaces.append(shape)
        if layer_dict['type'] == 0 or layer_dict['type'] == -1: # 0 is vacuum, -1 is vacuum vessel
            shape.name = shape.name + ' flagged for removal' # flagging vacuum components for removal from model
        if 'material' in layer_dict:
            material_tag_dict[layer_dict['name']]=layer_dict['material']

paramak_surfaces = [x for _,x in sorted(zip(max_r_coordinate_list, paramak_surfaces))]  # sort surfaces by maximum r value
min_r_coordinate_list = [x for _,x in sorted(zip(max_r_coordinate_list, min_r_coordinate_list))] # sort min r values to match

if 'flagged for removal' in paramak_surfaces[-1].name:  #remove outermost surface if it's vacuum
    paramak_surfaces.pop(-1)


cut_dictionary = {}  # initialize dictionary for surfaces to be cut
for surface in paramak_surfaces:
    print(surface.name)
    cut_dictionary[surface.name]=[]
    if surface.name == 'hfs TF':
        print('Adding ' + surface.name + ' to cut list for OH')
        cut_dictionary['OH'].append(surface)

i=0
while i<len(paramak_surfaces): #creates dictionary to define which surfaces to cut
    j=i+1
    current_surface = paramak_surfaces[i]
    while j<len(paramak_surfaces):
        # if current_surface.name == 'OH':
            # print('Adding ' + deepcopy(current_surface.name) + ' to cut list for hfs tf')
            # cut_dictionary['OH'].append(current_surface)
            # cut_dictionary['hfs shield'].append(current_surface)
        if min_r_coordinate_list[i]>=min_r_coordinate_list[j]: #checking mins because already sorted by maxes
            print('Adding ' + deepcopy(current_surface.name) + ' to cut list for ' + paramak_surfaces[j].name)
            cut_dictionary[paramak_surfaces[j].name].append(current_surface)
            break
        else:
            j+=1
    i+=1
print(cut_dictionary)
components = []
for surface in paramak_surfaces:   # cuts applicable surfaces from components
    component = deepcopy(surface)
    component.cut = cut_dictionary[surface.name]
    if 'flagged for removal' not in component.name:   # exports to component list if 'flagged for removal' not in the name
        component.color = (random.uniform(0.5,1), random.uniform(0.5,1), random.uniform(0.5,1))
        components.append(component)
        # if component.name == 'hfs TF':
        #     hfs_TF_object = component

component_volume_dictionary={}
for component in components:
    component_volume_dictionary[component.name.lower()]=component.volume()

reactor = paramak.Reactor(    # set up the reactor
    shapes_and_components=components
)
stp_filenames = reactor.export_stp(units='cm') # export the components
reactor.export_stp(filename='reactor.stp', units='cm') # export the reactor

Enter filepath for JSON: 
gap oh max r: 132.93021602952584
gap oh min r: 0.0
oh max r: 199.39532404428877
oh min r: 132.93021602952584
hfs tf max r: 1171.939872257583
hfs tf min r: 199.39532404428877
gap hfs tf shield max r: 1105.484258252073
gap hfs tf shield min r: 265.8604320590517
hfs shield max r: 1038.975817944273
hfs shield min r: 265.9105597988988
hfs blanket max r: 1005.7432639368917
hfs blanket min r: 299.14311380628027
hfs wall max r: 872.8563802004027
hfs wall min r: 365.60822182104323
vacuum vessel max r: 839.6093758146732
vacuum vessel min r: 398.8407758284247
gap cryostat max r: 1371.3446903111249
gap cryostat min r: 0.0
gap OH flagged for removal
OH
vacuum vessel flagged for removal
hfs wall
hfs blanket
hfs shield
gap hfs TF shield flagged for removal
hfs TF
Adding hfs TF to cut list for OH
Adding vacuum vessel flagged for removal to cut list for hfs wall
Adding hfs wall to cut list for hfs blanket
Adding hfs blanket to cut list for hfs shield
Adding hfs shield to cut l

['reactor.stp']

In [25]:
json_filename = 'tests/D3D.json'
json_file = open(json_filename)
data = json.load(json_file)
heights_list = []
widths_list = []
r_coordinates = []
z_coordinates = []
for coil in data['pf_active']['coil']:
    for key, rectangle in coil['element'][0]['geometry'].items():
        heights_list.append(rectangle['height'] * 100)
        widths_list.append(rectangle['width'] * 100)
        r_coordinates.append(rectangle['r'] * 100)
        z_coordinates.append(rectangle['z'] * 100)
center_points_list = list(zip(r_coordinates, z_coordinates))
print(heights_list)
print(widths_list)
print(center_points_list)
pf_coil_set = paramak.PoloidalFieldCoilSet(
    heights=heights_list,
    widths=widths_list,
    center_points=center_points_list,
    rotation_angle = 180,
)
pf_coil_set.export_stp('pf_set.stp', units='cm')

[26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 26.54246584113392, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275]
[16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 16.683333333333326, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275, 21.043212804043275]
[(55.6111111111111, -134.15098930165524), (55.6111111111111, -104.51901728768182), (55.6111111111111, -74.88704527370841), (55.6111111111111, -45.25507325973499), (55.6111111111111, -15.62310124576156), (55.6111111111111, 14.008870768211889), (55.6111111111111, 43.640842782185295), (55.6

'pf_set.stp'

In [ ]:
print(component_volume_dictionary)

In [ ]:
lithium_lead = nmm.Material.from_library(name='lithium-lead', temperature=293)
flibe = nmm.Material.from_library(name='FLiBe', temperature = 293)

material_tag_to_material_dict = {
    'OH':'ReBCO',
    'hfs wall':'Steel, Stainless 316L',
    'hfs blanket':lithium_lead,
    'hfs shield':'Steel, Stainless 316L',
    'hfs TF':'ReBCO',
}

# material_tag_to_material_dict = {
#     'OH':'Air (dry, near sea level)',
#     'hfs wall':'Air (dry, near sea level)',
#     'hfs blanket':'Air (dry, near sea level)',
#     'hfs shield':'Air (dry, near sea level)',
#     'hfs TF':'Air (dry, near sea level)',
# }

files_with_tags = [{'cad_filename':stp_filename, 'material_tag':name} for name, stp_filename in zip(reactor.name, stp_filenames)]

cad_to_h5m(
    files_with_tags = files_with_tags,
    h5m_filename = 'dagmc.h5m',
    cubit_path = '/home/mclaughlink/cubit_bld/Cubit-15.6/bin',
)

In [ ]:
settings = openmc.Settings()
settings.run_mode = 'fixed source'
num_of_batches = 2
settings.batches = num_of_batches
#settings.statepoint = {'batches': range(50, num_of_batches+50, 50)}
settings.inactive = 0
settings.particles = 5000
settings.photon_transport = True
settings.trigger_active=True
settings.trigger_max_batches = num_of_batches * 100
settings.trigger_batch_interval = 1
# settings.survival_biasing = True

ring_source_radius = (max_r_coordinate_dict['vacuum vessel'] + min_r_coordinate_dict['vacuum vessel'])/2
plasma_source = openmc_plasma_source.FusionRingSource(
    radius=ring_source_radius,
    angles=(0.0, rotation_angle*math.pi/180.0),
    fuel = 'DT'  
    # ADD TEMPERATURE
)
settings.source = plasma_source

In [ ]:
vol_number_to_cell_dict = di.get_volumes_and_materials_from_h5m('dagmc.h5m')
print(vol_number_to_cell_dict)

In [ ]:
tallies = openmc.Tallies()

# mesh
mesh = openmc.RegularMesh(mesh_id=1)
mesh.dimension = [50,50,50]
mesh.lower_left =  [-round(max(max_r_coordinate_list),-2), -round(max(max_r_coordinate_list),-2), -round(max(max_r_coordinate_list),-2)]
mesh.upper_right = [round(max(max_r_coordinate_list),-2), round(max(max_r_coordinate_list),-2), round(max(max_r_coordinate_list),-2)]

# filters
mesh_filter = openmc.MeshFilter(mesh, filter_id = 999)
energy_bins_n, dose_coeffs_n = openmc.data.dose_coefficients(particle='neutron', geometry='ISO')
energy_bins_p, dose_coeffs_p = openmc.data.dose_coefficients(particle='photon', geometry='ISO')
energy_function_filter_n = openmc.EnergyFunctionFilter(energy_bins_n, dose_coeffs_n, filter_id=998)
energy_function_filter_p = openmc.EnergyFunctionFilter(energy_bins_p, dose_coeffs_p, filter_id=997)
particle_filter_n = openmc.ParticleFilter('neutron', filter_id=996)
particle_filter_p = openmc.ParticleFilter('photon', filter_id=995)
energy_bins = openmc.mgxs.GROUP_STRUCTURES['CCFE-709']
energy_filter = openmc.EnergyFilter(energy_bins, filter_id=994)

# triggers
oh_heating_trigger = openmc.Trigger(trigger_type='rel_err', threshold=0.1)

pf_coil_filter_dict = {}
tf_coil_list = []
for key in vol_number_to_cell_dict:
    if vol_number_to_cell_dict[key].lower()=='hfs blanket':
        blanket_cell_filter = openmc.CellFilter([key],filter_id=key)
    elif vol_number_to_cell_dict[key].lower()=='hfs wall':
        wall_cell_filter = openmc.CellFilter([key],filter_id=key)
    elif vol_number_to_cell_dict[key].lower()=='hfs shield':
        shield_cell_filter = openmc.CellFilter([key],filter_id=key)
    elif vol_number_to_cell_dict[key].lower()=='oh':
        oh_cell_filter = openmc.CellFilter([key],filter_id=key)
    elif vol_number_to_cell_dict[key].lower()=='hfs tf':
        tf_coil_list.append(key)
tf_coil_cell_filter = openmc.CellFilter(tf_coil_list, filter_id=1000)

mesh_flux_tally_n = openmc.Tally(tally_id=999, name='Neutron Total Flux')
mesh_flux_tally_n.filters = [mesh_filter, particle_filter_n]
mesh_flux_tally_n.scores = ['flux']
tallies.append(mesh_flux_tally_n)

mesh_flux_tally_p = openmc.Tally(tally_id=998, name='Photon Total Flux')
mesh_flux_tally_p.filters = [mesh_filter, particle_filter_p]
mesh_flux_tally_p.scores = ['flux']
tallies.append(mesh_flux_tally_p)

dose_tally_neutron = openmc.Tally(tally_id=997, name='Neutron Dose')
dose_tally_neutron.scores = ['flux']
dose_tally_neutron.filters = [mesh_filter, particle_filter_n, energy_function_filter_n]
tallies.append(dose_tally_neutron)

dose_tally_photon = openmc.Tally(tally_id=996, name='Photon Dose')
dose_tally_photon.scores = ['flux']
dose_tally_photon.filters = [mesh_filter, particle_filter_p, energy_function_filter_p]
tallies.append(dose_tally_photon)

tf_coil_heating_tally = openmc.Tally(tally_id=tf_coil_cell_filter.id, name='TF Coil Heating')
tf_coil_heating_tally.filters = [tf_coil_cell_filter]
tf_coil_heating_tally.scores = ['heating']
tallies.append(tf_coil_heating_tally)

oh_heating_tally = openmc.Tally(tally_id=oh_cell_filter.id, name='CS Coil Heating')
oh_heating_tally.filters = [oh_cell_filter]
oh_heating_tally.scores = ['heating']
oh_heating_tally.triggers = [oh_heating_trigger]
tallies.append(oh_heating_tally)

wall_heating_tally = openmc.Tally(tally_id=wall_cell_filter.id, name='Wall Heating')
wall_heating_tally.filters = [wall_cell_filter]
wall_heating_tally.scores = ['heating']
tallies.append(wall_heating_tally)

shield_heating_tally = openmc.Tally(tally_id=shield_cell_filter.id, name='Shield Heating')
shield_heating_tally.filters = [shield_cell_filter]
shield_heating_tally.scores = ['heating']
tallies.append(shield_heating_tally)

blanket_heating_tally = openmc.Tally(tally_id=blanket_cell_filter.id, name='Blanket Heating')
blanket_heating_tally.filters = [blanket_cell_filter]
blanket_heating_tally.scores = ['heating']
tallies.append(blanket_heating_tally)

tf_coil_heating_tally_local = openmc.Tally(tally_id=tf_coil_cell_filter.id+100, name='TF Coil Heating Local')
tf_coil_heating_tally_local.filters = [tf_coil_cell_filter]
tf_coil_heating_tally_local.scores = ['heating-local']
tallies.append(tf_coil_heating_tally_local)

oh_heating_tally_local = openmc.Tally(tally_id=oh_cell_filter.id+100, name='CS Coil Heating Local')
oh_heating_tally_local.filters = [oh_cell_filter]
oh_heating_tally_local.scores = ['heating-local']
tallies.append(oh_heating_tally_local)

wall_heating_tally_local = openmc.Tally(tally_id=wall_cell_filter.id+100, name='Wall Heating Local')
wall_heating_tally_local.filters = [wall_cell_filter]
wall_heating_tally_local.scores = ['heating-local']
tallies.append(wall_heating_tally_local)

shield_heating_tally_local = openmc.Tally(tally_id=shield_cell_filter.id+100, name='Shield Heating Local')
shield_heating_tally_local.filters = [shield_cell_filter]
shield_heating_tally_local.scores = ['heating-local']
tallies.append(shield_heating_tally_local)

blanket_heating_tally_local = openmc.Tally(tally_id=blanket_cell_filter.id+100, name='Blanket Heating Local')
blanket_heating_tally_local.filters = [blanket_cell_filter]
blanket_heating_tally_local.scores = ['heating-local']
tallies.append(blanket_heating_tally_local)

blanket_TBR_tally = openmc.Tally(tally_id=blanket_cell_filter.id+200, name='Blanket TBR')
blanket_TBR_tally.filters = [blanket_cell_filter]
blanket_TBR_tally.scores = ['(n,Xt)']
tallies.append(blanket_TBR_tally)

blanket_flux_spectrum_tally = openmc.Tally(tally_id=blanket_cell_filter.id+300, name='Blanket Flux Spectrum')
blanket_flux_spectrum_tally.filters = [blanket_cell_filter, particle_filter_n, energy_filter]
blanket_flux_spectrum_tally.scores = ['flux']
tallies.append(blanket_flux_spectrum_tally)

geometry = odw.Geometry(h5m_filename='dagmc.h5m')
materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)
materials.cross_sections = '/fusion/projects/codes/neutronics/xs_libraries/endfb80_hdf5/cross_sections.xml'

In [ ]:
materials.export_to_xml()
settings.export_to_xml()
tallies.export_to_xml()
geometry.export_to_xml()
model=openmc.model.Model(geometry, materials, settings, tallies)

In [ ]:
!rm summary.h5
# !rm statepoint.10.h5
sp_filename = model.run()

In [8]:
sp = openmc.StatePoint('tests/statepoint.002.h5')

tf_coil_heating=sp.get_tally(name='TF Coil Heating')
cs_coil_heating=sp.get_tally(name='CS Coil Heating')
wall_heating=sp.get_tally(name='Wall Heating')
shield_heating=sp.get_tally(name='Shield Heating')
blanket_heating=sp.get_tally(name='Blanket Heating')
tbr_tally=sp.get_tally(name='Blanket TBR')
tf_coil_heating_local=sp.get_tally(name='TF Coil Heating Local')
cs_coil_heating_local=sp.get_tally(name='CS Coil Heating Local')
wall_heating_local=sp.get_tally(name='Wall Heating Local')
shield_heating_local=sp.get_tally(name='Shield Heating Local')
blanket_heating_local=sp.get_tally(name='Blanket Heating Local')
tbr_df = tbr_tally.get_pandas_dataframe()

tf_coil_heating_result, tf_coil_heating_st_dev = otuc.process_tally(
    tally=tf_coil_heating,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs tf']
    )
oh_coil_heating_result, oh_could_heating_st_dev = otuc.process_tally(
    tally=cs_coil_heating,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['oh']
)
wall_heating_result, wall_heating_st_dev = otuc.process_tally(
    tally=wall_heating,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs wall']
)
shield_heating_result, shield_heating_st_dev = otuc.process_tally(
    tally=shield_heating,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs shield']
)
blanket_heating_result, blanket_heating_st_dev = otuc.process_tally(
    tally=blanket_heating,
    required_units = 'joule / centimeter **3',
    volume=component_volume_dictionary['hfs blanket']
)
tbr_tally_result = tbr_df['mean'].sum()
tbr_tally_std_dev = tbr_df['std. dev.'].sum()

tf_coil_heating_result_local, tf_coil_heating_st_dev_local = otuc.process_tally(
    tally=tf_coil_heating_local,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs tf']
    )
oh_coil_heating_result_local, oh_could_heating_st_dev_local = otuc.process_tally(
    tally=cs_coil_heating_local,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['oh']
)
wall_heating_result_local, wall_heating_st_dev_local = otuc.process_tally(
    tally=wall_heating_local,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs wall']
)
shield_heating_result_local, shield_heating_st_dev_local = otuc.process_tally(
    tally=shield_heating_local,
    required_units='joule / centimeter **3',
    volume=component_volume_dictionary['hfs shield']
)
blanket_heating_result_local, blanket_heating_st_dev_local = otuc.process_tally(
    tally=blanket_heating_local,
    required_units = 'joule / centimeter **3',
    volume=component_volume_dictionary['hfs blanket']
)


# Find average heating on tf coils, find the st deviation of the average
tf_coil_heating_avg = sum(tf_coil_heating_result)/len(tf_coil_heating_result)
print(tf_coil_heating_avg)
print(tf_coil_heating_st_dev)
under_sqrt_list = []
for index, st_dev in enumerate(tf_coil_heating_st_dev):
    if tf_coil_heating_result[index] != 0:
        under_sqrt_list.append((st_dev/tf_coil_heating_result[index])**2)
print(under_sqrt_list)
tf_coil_heating_avg_rel_error = math.sqrt(sum(under_sqrt_list))/len(under_sqrt_list)
tf_coil_heating_avg_st_dev = tf_coil_heating_avg_rel_error*tf_coil_heating_avg

# print('TF Coil Heating: ', tf_coil_heating_result, '+/-', tf_coil_heating_st_dev) #WRONG
# print('OH Coil Heating: ', oh_coil_heating_result, '+/-', oh_could_heating_st_dev)
# print('Wall Heating: ', wall_heating_result, '+/-', wall_heating_st_dev)
# print('Shield Heating: ', shield_heating_result, '+/-', shield_heating_st_dev)
# print('Blanket Heating: ', blanket_heating_result,'+/-', blanket_heating_st_dev)
# print('TBR: ', tbr_tally_result, '+/-', tbr_tally_std_dev)

results_dict = {'Tally':['TF Coil Heating', 'OH Coil Heating', 'Wall Heating', 'Shield Heating', 'Blanket Heating', 'TBR', 'OH Coil Heating Local', 'Wall Heating Local', 'Shield Heating Local', 'Blanket Heating Local'],
                'Result (per source particle)':[tf_coil_heating_avg, oh_coil_heating_result, wall_heating_result, shield_heating_result, blanket_heating_result, tbr_tally_result,
                oh_coil_heating_result_local, wall_heating_result_local, shield_heating_result_local, blanket_heating_result_local],
                'St. Dev. (per source particle)':[tf_coil_heating_avg_st_dev, oh_could_heating_st_dev, wall_heating_st_dev, shield_heating_st_dev, blanket_heating_st_dev, tbr_tally_std_dev,
                oh_could_heating_st_dev_local, wall_heating_st_dev_local, shield_heating_st_dev_local, blanket_heating_st_dev_local]}
results_df = pandas.DataFrame(data=results_dict)
display(results_df)

blanket_neutron_spectrum_result=sp.get_tally(name='Blanket Flux Spectrum')

plot_spectrum_from_tally(
    spectrum={'Blanket Neutron Flux Spectrum':blanket_neutron_spectrum_result},
    x_label=  'Energy [eV]',
    y_label= 'Neutrons/Source Neutron/centimeter **2',
    plotting_package='plotly',
    required_units = 'neutrons / source_particle / centimeter **2',
    volume = component_volume_dictionary['hfs blanket']
)

7.492390050821692e-26 joule / centimeter ** 3
[7.285448015024812e-28 0.0 1.895350965857032e-26 1.696161295502721e-27 2.1354142889642527e-25 2.7759246683492192e-25 8.881350109105491e-26 1.1689775460579776e-26 1.1237691803821768e-25 3.306410717684307e-25 0.0 0.0 0.0 1.1778525628086404e-25 0.0 0.0] joule / centimeter ** 3
[<Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(0.801620055, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]


/home/mclaughlink/anaconda3/envs/neutronics/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



,Tally,Result (per source particle),St. Dev. (per source particle)
0,TF Coil Heating,7.492390050821692e-26 joule / centimeter ** 3,2.345682945909564e-26 joule / centimeter ** 3
1,OH Coil Heating,[0.0 joule / centimeter ** 3],[0.0 joule / centimeter ** 3]
2,Wall Heating,[6.173930051353379e-21 joule / centimeter ** 3],[3.616392918263189e-23 joule / centimeter ** 3]
3,Shield Heating,[3.308915351099463e-23 joule / centimeter ** 3],[9.01188339436076e-24 joule / centimeter ** 3]
4,Blanket Heating,[1.950790554800058e-22 joule / centimeter ** 3],[2.2914458271943863e-24 joule / centimeter ** 3]
5,TBR,0.193597,0.002779
6,OH Coil Heating Local,[0.0 joule / centimeter ** 3],[0.0 joule / centimeter ** 3]
7,Wall Heating Local,[6.1707688458516634e-21 joule / centimeter ** 3],[5.1679350173451726e-24 joule / centimeter ** 3]
8,Shield Heating Local,[3.1543883134534884e-23 joule / centimeter ** 3],[6.163059339397604e-24 joule / centimeter ** 3]
9,Blanket Heating Local,[1.651623869262562e-22 joule / centimeter ** 3],[9.939904368633138e-25 joule / centimeter ** 3]


/home/mclaughlink/anaconda3/envs/neutronics/lib/python3.7/site-packages/spectrum_plotter/core.py:288: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/home/mclaughlink/anaconda3/envs/neutronics/lib/python3.7/site-packages/spectrum_plotter/core.py:289: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/home/mclaughlink/anaconda3/envs/neutronics/lib/python3.7/site-packages/spectrum_plotter/core.py:291: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



In [ ]:
# all_materials = nmm.AvailableMaterials()
# for key in all_materials.keys():
#     print(key)